In [63]:
# Load ground truth
ground_truth = pd.read_csv('./test_data/midlevel.chunks_90.split_0.train.csv')
num_rows = ground_truth.shape[0]

# Print the result
print(f"Number of rows in the ground truth dataset: {num_rows}")
# Filter the dataset for the specific file_id
filtered_df = ground_truth[ground_truth['file_id'] == "vp1/run1b_2018-05-29-14-02-47.kinect_color"]

num_rows2 = filtered_df.shape[0]
print(f"Number of rows in the ground truth dataset having video run1b_2018-05-29-14-02-47.kinect_color : {num_rows2}")

filtered_df.head(10)

Number of rows in the ground truth dataset: 6642
Number of rows in the ground truth dataset having video run1b_2018-05-29-14-02-47.kinect_color : 363


,participant_id,file_id,annotation_id,frame_start,frame_end,activity,chunk_id
0,1,vp1/run1b_2018-05-29-14-02-47.kinect_color,1,58,82,closing_door_outside,0
1,1,vp1/run1b_2018-05-29-14-02-47.kinect_color,3,102,130,opening_door_outside,0
2,1,vp1/run1b_2018-05-29-14-02-47.kinect_color,4,130,156,entering_car,0
3,1,vp1/run1b_2018-05-29-14-02-47.kinect_color,5,156,174,closing_door_inside,0
4,1,vp1/run1b_2018-05-29-14-02-47.kinect_color,6,174,219,fastening_seat_belt,0
5,1,vp1/run1b_2018-05-29-14-02-47.kinect_color,6,219,230,fastening_seat_belt,1
6,1,vp1/run1b_2018-05-29-14-02-47.kinect_color,7,230,265,using_multimedia_display,0
7,1,vp1/run1b_2018-05-29-14-02-47.kinect_color,8,2985,3010,sitting_still,0
8,1,vp1/run1b_2018-05-29-14-02-47.kinect_color,9,3010,3055,pressing_automation_button,0
9,1,vp1/run1b_2018-05-29-14-02-47.kinect_color,10,3055,3101,sitting_still,0


In [57]:

# ✅ Group by annotation_id and merge frame ranges for the same activity
segmentedGroundTruth_df = filtered_df.groupby(["annotation_id", "activity"]).agg({
    "frame_start": "min",  # Get the earliest frame in the chunk
    "frame_end": "max",    # Get the latest frame in the chunk
    "chunk_id": lambda x: ", ".join(map(str, sorted(x)))  # Combine chunk_ids
}).reset_index()

# ✅ Sort data by annotation_id for clarity
segmentedGroundTruth_df = segmentedGroundTruth_df.sort_values(by=["annotation_id", "frame_start"])

# ✅ Save or display the segmented file
segmentedGroundTruth_df.to_csv("segmented_midpoint_file.csv", index=False)


In [64]:
# Parse the predictions.log file
predictions = []
with open('predictions.log', 'r') as file:
    for line in file:
        try:
            # Split the line to extract frame and activity information
            parts = line.strip().split(' - ')  # Splitting by " - " to separate timestamp and frame info
            frame_part = parts[1].split(': ')[0]  # Extract "Frame X"
            frame = int(frame_part.split()[1])  # Extract the frame number after "Frame"

            # Extract activity and confidence
            activity_part = parts[1].split(': ', maxsplit=1)[1]  # This contains "Predicted Activity: XYZ, Confidence: XX.XX%"
            activity = activity_part.split(', Confidence')[0].replace("Predicted Activity: ", "").strip()  # Extract actual activity
            
            # Extract confidence if it exists
            confidence = 0.0  # Default confidence
            if "Confidence" in activity_part:  # Check if "Confidence" exists in the string
                confidence = float(activity_part.split('Confidence: ')[1].strip('%'))  # Extract confidence as float

            # Append to predictions list
            predictions.append({'frame': frame, 'activity': activity, 'confidence': confidence})
        
        except (IndexError, ValueError) as e:
            print(f"Skipping line due to error: {line.strip()} -> {e}")

# Convert predictions to DataFrame
predictions_df = pd.DataFrame(predictions)

# Display the predictions DataFrame
print(predictions_df.head(10))

   frame              activity  confidence
0      1  opening_door_outside       54.08
1      2  opening_door_outside       57.44
2      3  opening_door_outside       57.44
3      4  opening_door_outside       57.44
4      5  opening_door_outside       57.44
5      6  opening_door_outside       57.44
6      7  opening_door_outside       57.44
7      8  opening_door_outside       57.44
8      9  opening_door_outside       58.80
9     10  opening_door_outside       58.80


In [65]:
# Define missing frame range (inclusive)
missing_intervals = [(266, 2984)]  # Adjust if needed

# Function to check if a frame falls within the missing range (inclusive)
def is_in_missing_range(frame):
    return any(start <= frame <= end for start, end in missing_intervals)

# Drop frames within the missing range
filtered_predictions_df = predictions_df[~predictions_df['frame'].apply(is_in_missing_range)]
filtered_predictions_df.to_csv('filtered_predictions.csv', index=False)

In [66]:
#segment the predictions
import pandas as pd

def convert_predictions_to_segments(predictions):
    segments = []
    current_activity = None
    current_start = None

    for idx, row in predictions.iterrows():
        frame = row['frame']
        activity = row['activity']

        # Start a new segment if the activity changes
        if activity != current_activity:
            if current_activity is not None:
                # Save the previous segment
                segments.append({
                    'frame_start': current_start,
                    'frame_end': frame - 1,
                    'activity': current_activity
                })
            # Start a new segment
            current_activity = activity
            current_start = frame

    # Save the last segment
    if current_activity is not None:
        segments.append({
            'frame_start': current_start,
            'frame_end': predictions.iloc[-1]['frame'],
            'activity': current_activity
        })

    # Convert to DataFrame
    segments_df = pd.DataFrame(segments)

    # Drop the first row
    if not segments_df.empty:
        segments_df = segments_df.iloc[1:].reset_index(drop=True)

    return segments_df

# Example usage
segments_df = convert_predictions_to_segments(filtered_predictions_df)

# Display the modified segments DataFrame
print(segments_df.head(10))
segments_df.to_csv('segmented_activities.csv', index=False)

# weird segment (240       2984    using_multimedia_display )

   frame_start  frame_end                    activity
0           55         90        closing_door_outside
1           91        136        opening_door_outside
2          137        157                entering_car
3          158        184         closing_door_inside
4          185        239         fastening_seat_belt
5          240       2984    using_multimedia_display
6         2985       3007               sitting_still
7         3008       3056  pressing_automation_button
8         3057       3116               sitting_still
9         3117       3154    using_multimedia_display


In [67]:
# unique activities
import pandas as pd

# Extract the unique activities from the 'activity' column
unique_activities = ground_truth['activity'].dropna().unique()

# Convert to a list
unique_activities_list = list(unique_activities)

# Print the unique activities
print(unique_activities_list)
len(unique_activities_list)

['closing_door_outside', 'opening_door_outside', 'entering_car', 'closing_door_inside', 'fastening_seat_belt', 'using_multimedia_display', 'sitting_still', 'pressing_automation_button', 'fetching_an_object', 'opening_laptop', 'working_on_laptop', 'interacting_with_phone', 'closing_laptop', 'placing_an_object', 'unfastening_seat_belt', 'putting_on_jacket', 'opening_bottle', 'drinking', 'closing_bottle', 'looking_or_moving_around (e.g. searching)', 'preparing_food', 'eating', 'taking_off_sunglasses', 'putting_on_sunglasses', 'reading_newspaper', 'writing', 'talking_on_phone', 'reading_magazine', 'taking_off_jacket', 'opening_door_inside', 'exiting_car', 'opening_backpack', 'putting_laptop_into_backpack', 'taking_laptop_from_backpack']


34

In [68]:
# Extract unique activities from the dataset
dataset_activities = set(filtered_df['activity'].unique())

# Find missing activities
missing_activities = [activity for activity in unique_activities_list if activity not in dataset_activities]

# Print the missing activities
print("Activities missing from the dataset:", missing_activities)

Activities missing from the dataset: ['opening_backpack', 'putting_laptop_into_backpack', 'taking_laptop_from_backpack']


In [78]:
import pandas as pd

def evaluate_multiclass(filtered_df, segmented_df, activity_classes):
    """
    Evaluates activity predictions using the Midpoint Hit Criterion.

    Fixes:
    - Ensures `metrics` includes all activities from `filtered_df` and `segmented_df`.
    - Prevents KeyError for missing activities.
    - Matches predictions using midpoint hit logic.
    - Handles False Positives and False Negatives correctly.

    Args:
    - filtered_df (pd.DataFrame): Ground truth dataframe with 'frame_start', 'frame_end', 'activity', 'annotation_id', 'chunk_id'.
    - segmented_df (pd.DataFrame): Predicted segments dataframe with 'frame_start', 'frame_end', 'activity'.
    - activity_classes (list): List of valid activity classes from ground truth.

    Returns:
    - precision, recall, overall_precision, overall_recall
    """

    #  Fix: Include all activities from both ground truth and predictions
    all_activities = set(filtered_df['activity']).union(set(segmented_df['activity']))

    # Initialize metrics for all known activities
    metrics = {cls: {'tp': 0, 'fp': 0, 'fn': 0} for cls in all_activities}
    matched_chunks = set()  # Tracks matched ground truth chunks (annotation_id, chunk_id)

    #  Iterate through ground truth activities in `filtered_df`
    for _, gt in filtered_df.iterrows():
        gt_start = gt['frame_start']
        gt_end = gt['frame_end']
        gt_activity = gt['activity']
        chunk_key = (gt['annotation_id'], gt['chunk_id'])

        #  Compute the exact midpoint of the ground truth segment
        gt_midpoint = (gt_start + gt_end) // 2

        #  Find prediction that matches the midpoint
        matched_prediction = segmented_df[
            (segmented_df['frame_start'] <= gt_midpoint) &
            (segmented_df['frame_end'] >= gt_midpoint) &
            (segmented_df['activity'] == gt_activity)
        ]

        if not matched_prediction.empty:
            #  True Positive: A prediction exists for this midpoint
            if chunk_key not in matched_chunks:
                metrics[gt_activity]['tp'] += 1  # Count as TP
                matched_chunks.add(chunk_key)  # Prevent duplicate matches
            else:
                metrics[gt_activity]['fp'] += 1  # False Positive for duplicates
        else:
            # False Negative: No correct prediction found for this activity midpoint
            metrics[gt_activity]['fn'] += 1

    # Count False Positives for unmatched predictions
    for _, pred in segmented_df.iterrows():
        pred_activity = pred['activity']

        #  Ignore if the activity was already correctly matched
        if pred_activity in metrics and metrics[pred_activity]['tp'] > 0:
            continue

        #  Fix: Ensure `pred_activity` exists in `metrics` before updating
        if pred_activity not in metrics:
            continue  # Ignore activities that are not in ground truth

        #  False Positive: No corresponding ground truth found for this prediction
        metrics[pred_activity]['fp'] += 1

    #  Compute Precision & Recall for each activity
    precision, recall = {}, {}
    for cls in all_activities:
        tp, fp, fn = metrics[cls]['tp'], metrics[cls]['fp'], metrics[cls]['fn']
        precision[cls] = tp / (tp + fp) * 100 if (tp + fp) > 0 else 0
        recall[cls] = tp / (tp + fn) * 100 if (tp + fn) > 0 else 0

    #  Compute Overall Precision & Recall
    overall_tp = sum(metrics[cls]['tp'] for cls in all_activities)
    overall_fp = sum(metrics[cls]['fp'] for cls in all_activities)
    overall_fn = sum(metrics[cls]['fn'] for cls in all_activities)

    overall_precision = overall_tp / (overall_tp + overall_fp) * 100 if (overall_tp + overall_fp) > 0 else 0
    overall_recall = overall_tp / (overall_tp + overall_fn) * 100 if (overall_tp + overall_fn) > 0 else 0

    return precision, recall, overall_precision, overall_recall


# Example usage:
dataset_activities = set(filtered_df['activity'].unique())

# Remove missing activities from evaluation (Only keep ones that exist in ground truth)
filtered_activity_classes = [activity for activity in unique_activities_list if activity in dataset_activities]

# Run the evaluation
precision, recall, overall_precision, overall_recall = evaluate_multiclass(filtered_df, segments_df, filtered_activity_classes)

#  Print results
print("Precision per class (%):", {cls: f"{precision[cls]:.2f}%" for cls in precision})
print("Recall per class (%):", {cls: f"{recall[cls]:.2f}%" for cls in recall})
print(f"Overall Precision: {overall_precision:.2f}%")
print(f"Overall Recall: {overall_recall:.2f}%")


Precision per class (%): {'entering_car': '100.00%', 'looking_or_moving_around (e.g. searching)': '100.00%', 'opening_laptop': '100.00%', 'unfastening_seat_belt': '100.00%', 'talking_on_phone': '100.00%', 'taking_off_sunglasses': '100.00%', 'reading_magazine': '100.00%', 'taking_off_jacket': '100.00%', 'opening_backpack': '0.00%', 'placing_an_object': '100.00%', 'fastening_seat_belt': '100.00%', 'drinking': '100.00%', 'opening_door_outside': '100.00%', 'exiting_car': '100.00%', 'working_on_laptop': '100.00%', 'using_multimedia_display': '100.00%', 'reading_newspaper': '100.00%', 'opening_door_inside': '100.00%', 'sitting_still': '100.00%', 'eating': '100.00%', 'closing_door_outside': '100.00%', 'putting_on_jacket': '100.00%', 'putting_laptop_into_backpack': '0.00%', 'preparing_food': '100.00%', 'pressing_automation_button': '100.00%', 'closing_laptop': '100.00%', 'closing_bottle': '100.00%', 'taking_laptop_from_backpack': '0.00%', 'writing': '100.00%', 'closing_door_inside': '100.00%',